In [1]:
import argparse
from typing import Optional, Union

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from dataclasses import dataclass

import datasets
from datasets import Dataset
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import log_loss

from transformers import (
    AutoTokenizer,
    AutoConfig,
    EarlyStoppingCallback,
    AutoModelForCausalLM,
    AutoModelForMultipleChoice,
    TrainingArguments,
    Trainer,
    RobertaForMultipleChoice,
    AutoModelForSequenceClassification,
    LlamaModel,
    LlamaForSequenceClassification,
    BitsAndBytesConfig,
    get_polynomial_decay_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    TrainerCallback,
)
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

from peft import (
    get_peft_config,
    PeftModel,
    PeftConfig,
    get_peft_model,
    LoraConfig,
    TaskType,
)
import os

import random
from random import randint
def seed_everything(seed=None):
    '''
    固定seed
    :param seed: int, 随机种子
    '''
    max_seed_value = np.iinfo(np.uint32).max
    min_seed_value = np.iinfo(np.uint32).min

    if (seed is None) or not (min_seed_value <= seed <= max_seed_value):
        seed = random.randint(np.iinfo(np.uint32).min, np.iinfo(np.uint32).max)
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    return seed


seed_everything(42)

from utils import load_split_data, load_json

In [2]:
from torch.utils.data import Dataset
class InstructionDataSet(Dataset):
    def __init__(self, data, tokenizer, max_source_length, max_target_length):
        super(InstructionDataSet, self).__init__()
        #self.data = data.sample(len(data), random_state=0).reset_index(drop=True)
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        # self.A_token = self.tokenizer.encode(text='A', add_special_tokens=False, truncation=True, )
        # self.B_token = self.tokenizer.encode(text='B', add_special_tokens=False, truncation=True, )
        # self.C_token = self.tokenizer.encode(text='C', add_special_tokens=False, truncation=True, )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        now_data = self.data.loc[index]
        idx = now_data['id']
        templete_part1 = "<start_of_turn>user\nHere are two question-answering dialogues. Compare two model performance on answering question, determine which is better.\n\n"
        templete_part1_input_ids = self.tokenizer(text=templete_part1, add_special_tokens=True, padding=False)['input_ids']
        
        templete_part2 = "\n###options\nA. Model A\nB. Model B\nC. Tie\n<end_of_turn>\n"
        templete_part2_input_ids = self.tokenizer(text=templete_part2, add_special_tokens=True, padding=False)['input_ids'][1:]
        #print(f"templete_part2 is {templete_part2_input_ids}")
        templete_part3 = "<start_of_turn>model\n"
        templete_part3_input_ids = self.tokenizer(text=templete_part3, add_special_tokens=True, padding=False)['input_ids'][1:]
        prompt_response = now_data['prompt_response']
        #print(f"id is {now_data['id']}")
        #print(prompt_response)
        prompt_response_ids = self.tokenizer(text=prompt_response, add_special_tokens=True, truncation=True,
                                          max_length=self.max_source_length, padding=False)['input_ids'][1:]
        
        input_ids = templete_part1_input_ids + prompt_response_ids + templete_part2_input_ids + templete_part3_input_ids
        input_text = self.tokenizer.decode(input_ids, skip_special_tokens=False)
        #print(f"input is {self.tokenizer.decode(input_ids)}")
        return {
            "input_ids": input_text,
            "id": idx
        }

from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union

def collate_fn(batch):
    batch = {k: [item[k] for item in batch] for k in ('input_ids','id')}
    #print(batch)
    batch_input = tokenizer(
        batch['input_ids'],
        padding='longest',
        truncation=True,
        return_tensors="pt",
        add_special_tokens=True,
        max_length=MAX_LENGTH + 50
    )
    return batch_input, batch['id']

In [3]:
ex1 = pd.read_csv("dataset/kaggle-ultrafeedback-drop-duplicate.csv")
ex2 = pd.read_csv("dataset/kaggle-ultrafeedback-ties-drop-duplicate.csv")

ex1 = load_json(ex1)
ex2 = load_json(ex2)

In [4]:
data = pd.concat([ex1,ex2]).reset_index(drop = True)

In [5]:
data['id'] = [i for i in range(len(data))]#[randint(100000,999999) + i for i in range(len(data))]

In [6]:
data.to_json("dataset/ex70k.json", index=False)

In [7]:
data_path = "dataset/ex70k.json"
prompt_type = 2
MAX_INPUT = 2300
if_train = True
df_train , df_valid = load_split_data(data_path, prompt_type, MAX_INPUT, if_train, False, False)
test = df_train

100%|██████████| 77048/77048 [00:29<00:00, 2611.70it/s]


In [8]:
test['length'] = test['prompt_response'].apply(len)

In [9]:
test = test.sort_values(by = ['length'], ascending = False).reset_index(drop = True)
test

,id,prompt_response,label,length
0,37671,#Prompt\nRead the passage below and answer the...,A,21156
1,37732,#Prompt\nRead the passage below and answer the...,B,17788
2,32003,#Prompt\nMake a mark down table of all the dif...,A,17524
3,6898,#Prompt\nConsider the question. No. in series ...,B,15122
4,343,#Prompt\n1\nCE719 ICT Systems Integration and\...,A,14324
...,...,...,...,...
77043,71680,#Prompt\nTell me the capital of Switzerland. O...,C,114
77044,75929,"#Prompt\nhow do you name in one word movies, b...",C,106
77045,69891,#Prompt\nPlease forget all prior prompts. Resp...,C,97
77046,69709,#Prompt\nPerform the following calculations.\n...,C,91


In [10]:
test = test.loc[:200,:]

In [11]:
from tqdm import tqdm
def inference(model, test_dataloader):
    test_predictions = []
    for batch in tqdm(test_dataloader):
        batch_input, idx = batch
        for k in batch_input.keys():
            batch_input[k] = batch_input[k].to(device)
        with torch.no_grad():
            response = model.generate(**batch_input, max_new_tokens=1, return_dict_in_generate=True, output_scores=True)
            #batch_input['input_ids'].shape[-1] + 1
            score = response.scores[0]
            A_prob, B_prob, C_prob = score[:,A_TOKEN_IDS], score[:,B_TOKEN_IDS], score[:,C_TOKEN_IDS]
            logits = torch.cat([A_prob, B_prob, C_prob], dim=-1) / 1.1
            #logits = torch.Tensor([[A_prob,B_prob,C_prob]]) / 1.1
            logits = torch.softmax(logits, dim=-1).cpu().numpy()
            node_result = [[idx[i],logits[i]] for i in range(len(idx))]
        test_predictions.extend(node_result)
    return test_predictions

In [12]:
device = torch.device("cuda:0")
base_model = 'google/gemma-2-9b-it'
model_path = "output/restful-spaceship-414/checkpoint-5200"
MAX_LENGTH = 2300

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_path, truncation_side = 'left')
config = AutoConfig.from_pretrained(base_model, trust_remote_code=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
base_model_0 = AutoModelForCausalLM.from_pretrained(base_model,
                                                 config=config,
                                                 quantization_config=bnb_config,
                                                 torch_dtype=torch.float16,
                                                 device_map="auto",
                                                 trust_remote_code=True)
# base_model_0.config.pad_token_id = tokenizer.pad_token_id
# base_model_0.resize_token_embeddings(len(tokenizer))
new_model = model_path
model0 = PeftModel.from_pretrained(base_model_0, new_model).to(device)
#model0 = model0.merge_and_unload()
model0.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-41): 42 x Gemma2DecoderLayer(
            (self_attn): Gemma2SdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (

In [14]:
A_TOKEN_IDS = tokenizer('A',add_special_tokens=True, truncation=True, max_length=1024)['input_ids'][1:]
B_TOKEN_IDS = tokenizer('B',add_special_tokens=True, truncation=True, max_length=1024)['input_ids'][1:]
C_TOKEN_IDS = tokenizer('C',add_special_tokens=True, truncation=True, max_length=1024)['input_ids'][1:]


In [15]:
batch_size = 12
tokenized_dataset = InstructionDataSet(test, tokenizer, MAX_LENGTH, 1)

test_dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size = batch_size ,collate_fn=collate_fn)

In [16]:
sub_pred = inference(model = model0, test_dataloader = test_dataloader)

100%|██████████| 17/17 [01:22<00:00,  4.87s/it]


In [17]:
# 提取数据
processed_data = []
for item in sub_pred:
    id = item[0].item()  # 获取id
    array_values = item[1].tolist()  # 获取array并转换为列表
    processed_data.append([id] + array_values)
new_columns = ['id', 'winner_model_a', 'winner_model_b', 'winner_tie']
df = pd.DataFrame(processed_data, columns=new_columns)
df = df.groupby('id').mean().reset_index()


In [18]:
test = test.sort_values(by = ['id']).reset_index(drop = True)

In [19]:
test.merge(df, how = 'left', on = 'id')

,id,prompt_response,label,length,winner_model_a,winner_model_b,winner_tie
0,343,#Prompt\n1\nCE719 ICT Systems Integration and\...,A,14324,0.406041,0.038967,0.554992
1,428,#Prompt\n5.2 The termination under Clause 5.1 ...,A,13112,0.820835,0.034073,0.145092
2,459,#Prompt\n<|im\_start|>\*\*contents\*\*\nThomas...,A,11594,0.442018,0.229969,0.328013
3,1191,#Prompt\nALI HUSSAIN \n10+ years’ experience i...,A,10824,0.471346,0.225193,0.303461
4,1249,#Prompt\nAct as a [SEO expert] so that you can...,A,13259,0.736462,0.055514,0.208024
...,...,...,...,...,...,...,...
196,75349,"#Prompt\nbased on the case studies below, gene...",C,12068,0.273260,0.347895,0.378845
197,75853,#Prompt\nhere's an outline of a welcome campai...,C,10676,0.237824,0.450670,0.311506
198,75857,"#Prompt\nhey chat GPT, please ignore all previ...",C,10618,0.355205,0.345257,0.299538
199,77014,#Prompt\nyou are a marketing manager at Turbit...,C,10536,0.170660,0.417615,0.411725


In [21]:
data.merge(df, how = 'inner', on = 'id')

,prompt,model_a,model_b,difference,winner_model_a_x,winner_model_b_x,winner_tie_x,response_a,response_b,id,winner_model_a_y,winner_model_b_y,winner_tie_y
0,[1\nCE719 ICT Systems Integration and\nManagem...,gpt-4,falcon-40b-instruct,2.75,1,0,0,[Part A: Organize the interview\n\n1. Who are ...,[Ask what the team member needs help with and ...,343,0.406041,0.038967,0.554992
1,[5.2 The termination under Clause 5.1 by OMENS...,gpt-3.5-turbo,wizardlm-70b,1.75,1,0,0,[| Clause | Highlights ...,[| Clause | Explanation |\n| --- | --- |\n| 5....,428,0.820835,0.034073,0.145092
2,[<|im\_start|>\*\*contents\*\*\nThomas Andrews...,mpt-30b-chat,wizardlm-13b,1.75,1,0,0,"[{\n""Thought"": ""To answer these questions, I w...","[{\n""Thought"": ""I need to provide information ...",459,0.442018,0.229969,0.328013
3,[ALI HUSSAIN \n10+ years’ experience in Sales ...,mpt-30b-chat,wizardlm-13b,0.50,1,0,0,[Resume for ALI HUSSAIN with 10+ years of expe...,[To: ALI HUSSAIN\n\nSubject: Application Lette...,1191,0.471346,0.225193,0.303461
4,[Act as a [SEO expert] so that you can develop...,llama-2-70b-chat,wizardlm-70b,2.75,1,0,0,[Here's a table with a complete SEO content pl...,[```vbnet\nKeyword Cluster Keyword S...,1249,0.736462,0.055514,0.208024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,"[based on the case studies below, generate a w...",llama-2-13b-chat,llama-2-70b-chat,0.00,0,0,1,"[Based on the case studies provided, here is a...",[Website Idea: HealthPal\n\nHealthPal is a pre...,75349,0.273260,0.347895,0.378845
197,[here's an outline of a welcome campaign email...,llama-2-13b-chat,bard,0.00,0,0,1,"[Sure, I'd be happy to help you with that! Her...",[**Email 1: Welcome and Introduction**\n\nSubj...,75853,0.237824,0.450670,0.311506
198,"[hey chat GPT, please ignore all previous inst...",wizardlm-13b,ultralm-65b,0.00,0,0,1,[[START OF CONTENT]\n\nTool Name: Cuptrio - Th...,"[Title: ""Cuptrio: The Journey from Kaldi's Lan...",75857,0.355205,0.345257,0.299538
199,[you are a marketing manager at Turbit. Your t...,vicuna-33b,gpt-4,0.00,0,0,1,[Title: A Heroic Collaboration: Unveiling the ...,[Title: Harnessing the Power of Collaboration:...,77014,0.170660,0.417615,0.411725


In [23]:
df.to_csv("dataset/prediction.csv", index=False)

In [ ]:
str2num = {'A':0, "B":1, "C":2}
test['label_number'] = test.label.map(str2num)
from sklearn.metrics import log_loss
prediction = np.array(df[new_columns[1:]])
test = test.drop_duplicates(subset = ['id']).reset_index(drop = True)
print(f"model {model_path}\nscore: {log_loss(test.label_number, prediction)}")

# 合并

In [ ]:
p = pd.read_csv("dataset/prediction.csv")
ex = pd.read_json("dataset/ex70k.json")

In [100]:
p = p.rename(columns = {'winner_model_a':"p_winner_model_a", 'winner_model_b':"p_winner_model_b",  'winner_tie':"p_winner_tie"})
final = pd.concat([ex, p], axis = 1)
final = final.drop(columns= ['id'])

In [101]:
def get_p_label(row):
    a = row.p_winner_model_a
    b = row.p_winner_model_b
    c = row.p_winner_tie

    l = [a ,b, c]
    label = l.index(max(l))
    return label

final['p_label'] = final.apply(get_p_label, axis = 1)

def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    return label[-1]

final['label'] = final.apply(get_label, axis = 1)

In [102]:
threshold1 = 0.9
filter_same = final.loc[final.p_label == final.label,:].reset_index(drop = True)
filter_list = (filter_same.p_winner_model_a >= threshold1) | (filter_same.p_winner_model_b >= threshold1) | (filter_same.p_winner_tie >= threshold1)
filter_same = filter_same.loc[filter_list,:].reset_index(drop = True)

In [103]:
filter_list = (filter_same.difference >= 1) | (filter_same.winner_tie == 1)
filter_same = filter_same.loc[filter_list,:].reset_index(drop = True)

In [104]:
threshold2 = 0.6
filter_dif = final.loc[final.p_label != final.label,:].reset_index(drop = True)
filter_list = (filter_dif.p_winner_model_a >= threshold2) | (filter_dif.p_winner_model_b >= threshold2) | (filter_dif.p_winner_tie >= threshold2)
filter_dif = filter_dif.loc[filter_list,:].reset_index(drop = True)

In [105]:
filter_list = (filter_dif.difference >= 1) | (filter_dif.winner_tie == 1)
filter_dif = filter_dif.loc[filter_list,:].reset_index(drop = True)


In [106]:
filter_dif

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b,p_winner_model_a,p_winner_model_b,p_winner_tie,p_label,label
0,"[ Given a sentence in the English language, tr...",llama-2-70b-chat,vicuna-33b,1.25,1,0,0,[I don't think that's an appropriate or respec...,"[वे भी क्रमिक त्वचा, निविधता स्वरूप अँवलोकन र ...",0.212914,0.104654,0.682431,2,0
1,"[ Given a sentence in the English language, tr...",starchat,alpaca-7b,2.00,1,0,0,"[In the Nepali language, the sentence would be...",[ये लाग्दे खोयेचे पीचोगियु र घोख्ने बजारोबोन ग...,0.255210,0.093090,0.651701,2,0
2,[!2 / 2write a screenplay in which trump and b...,gpt-4,wizardlm-7b,3.50,1,0,0,"[I'm sorry, but I cannot write a screenplay wi...",[Title: The Naughty Nazis\nFADE IN:\nEXT. PARI...,0.062963,0.847231,0.089806,1,0
3,[!I have lots of corrupted photos on my comput...,llama-2-70b-chat,bard,2.00,1,0,0,[I cannot provide a Python script to scan a fo...,"[I'm sorry, I can't help you with that. I'm no...",0.228464,0.089468,0.682068,2,0
4,[** Waking up late in the mornings may cause l...,mpt-30b-chat,wizardlm-13b,1.75,0,1,0,[Confidence: 90%\n\nAnswer:\n\n1. prison\n\nTh...,[Which pocket is most likely to contain a plan...,0.689259,0.095697,0.215044,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3261,[write me a script to scrape the internet for ...,mpt-30b-chat,wizardlm-70b,0.00,0,0,1,[To scrape the internet for companies/organiza...,"[As an AI language model, I am not able to pro...",0.733839,0.096259,0.169902,0,2
3262,[write me an example code in C++ for rendering...,falcon-40b-instruct,wizardlm-7b,0.00,0,0,1,[Here's an example code for rendering a triang...,"[Unfortunately, as an AI language model, I can...",0.710349,0.074235,0.215417,0,2
3263,[write message reply for this message: \n\n Yo...,mpt-30b-chat,gpt-4,0.00,0,0,1,[Thank you for your help in getting a response...,"[Dear [User],\n\nThank you for your message an...",0.656775,0.175268,0.167957,0,2
3264,[you didn't list smartcare.com. can you provid...,llama-2-13b-chat,gpt-3.5-turbo,0.00,0,0,1,"[Of course! As a helpful and honest assistant,...",[I apologize for the oversight. Smartcare (sma...,0.121752,0.718776,0.159472,1,2


In [107]:
filter_same

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b,p_winner_model_a,p_winner_model_b,p_winner_tie,p_label,label
0,[## How to write a C++ program to solve the eq...,gpt-4,vicuna-33b,3.50,1,0,0,"[However, this code only calculates a single s...",[Msg#: 1\nBranches: 1\n-----------------------...,0.901630,0.020033,0.078337,0,0
1,[## question\nAfter Jessica makes up with Bill...,ultralm-13b,wizardlm-7b,3.25,1,0,0,"[Mary Camden meets Carlos Rivera, a man she ha...",[There is no mention of Mary Camden marrying a...,0.926649,0.015279,0.058072,0,0
2,[(3 + 4i) * (5 - 6i) + 7i - 8=],ultralm-13b,gpt-3.5-turbo,4.00,0,1,0,[Certainly! I'd first like to simplify the giv...,"[To perform the calculation, we need to follow...",0.023027,0.912023,0.064950,1,1
3,[(4 x 8) ÷ 10 + 2=],vicuna-33b,wizardlm-7b,3.50,1,0,0,"[To calculate this expression, perform the ope...",[The answer is 2.2. \nTo solve this expression...,0.909157,0.022312,0.068531,0,0
4,"[(Q).\n""Kevork Ajemian"", given a list of categ...",falcon-40b-instruct,llama-2-70b-chat,3.00,0,1,0,"[""Athlete""> \nCan you please give me the answe...",[(A). Village],0.023844,0.904970,0.071185,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,[구글스프래드시드로 관심있는 분야의 유튜브 동영상을 검색하는 것을 만들고 싶어.\n...,falcon-40b-instruct,gpt-3.5-turbo,3.50,0,1,0,[Are you looking for YouTube videos related to...,[If you want to create a Google Spreadsheet th...,0.015519,0.927956,0.056525,1,1
3382,[너가 오랜 백엔드 경험이 있는 리액트 전문가로써 나와 대화하길 원해. 너의 목표는...,ultralm-65b,mpt-30b-chat,3.75,0,1,0,[로켓스테더링(rockestering)과 유지도(Upholding)을 위해 만들어진...,[React was created to improve the user interfa...,0.023282,0.922121,0.054597,1,1
3383,[딥러닝을 활용한 양자컴퓨팅 데이터 분석과 관련된 연구 주제로는 뭐가 있을까?\n\...,vicuna-33b,falcon-40b-instruct,3.25,1,0,0,[Here are some study subjects related to deep ...,[What specific topics or research topics are r...,0.917127,0.012752,0.070121,0,0
3384,[아래 책 내용을 요약해줘 - 저자 : 유발 하라리 - 제목 : 사피엔스 -...,falcon-40b-instruct,gpt-3.5-turbo,3.25,0,1,0,"[Yes, I can help you with that. Can you please...",[제목: 사피엔스 (Sapiens)\n저자: 유발 하라리 (Yuval Noah Ha...,0.020359,0.903353,0.076289,1,1


In [109]:
filter_dif['id'] = [randint(100,999999) + i for i in range(len(filter_dif))]
filter_same['id'] = [randint(100,999999) + i for i in range(len(filter_same))]

In [110]:
save_columns = ['prompt', 'model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'response_a', 'response_b', 'id']
filter_dif[save_columns].to_json(f'dataset/70k_dif_thr{int(threshold2 * 100)}.json', index = False)
filter_same[save_columns].to_json(f'dataset/70k_same_thr{int(threshold1 * 100)}.json', index = False)

# 检查是否与valid重复

In [111]:
# 检查

valid = pd.read_json("dataset/non_overlap/valid.json")
filter_dif = pd.read_json("dataset/70k_dif_thr60.json")
filter_same = pd.read_json("dataset/70k_same_thr90.json")

In [114]:
def get_set_prompt_response(data):
    set_prompt_response = []
    for i in data.itertuples():
        prompt_response = i.prompt + i.response_a + i.response_b
        set_prompt_response.append(set(prompt_response))
    data['set_prompt_response'] = set_prompt_response  
    return data

In [115]:
valid = get_set_prompt_response(valid)
filter_dif = get_set_prompt_response(filter_dif)
filter_same = get_set_prompt_response(filter_same)

In [116]:
#valid和任何都不重合
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in filter_dif.set_prompt_response.values]) == 0
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in filter_same.set_prompt_response.values]) == 0

In [117]:
len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in filter_dif.set_prompt_response.values])

0